In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ⚙️ Step 1: Install dependencies
!pip install -q sentence-transformers faiss-cpu

# ⚠️ If you are using Colab Pro+ and want FAISS-GPU:
# !pip install -q faiss-gpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00


In [4]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
import os, pickle, gc
from tqdm import tqdm
import torch

# 🔧 CONFIG
DATA_PATH = '/content/drive/MyDrive/kg_triplets.csv'
BATCH_SIZE = 512
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
FAISS_INDEX_PATH = '/content/drive/MyDrive/kg_faiss_index.faiss'
METADATA_PATH = '/content/drive/MyDrive/kg_faiss_metadata.pkl'

# ✅ Load model
model = SentenceTransformer(MODEL_NAME).to('cuda')
print("✅ Model loaded.")

# ✅ Load KG triplets
df = pd.read_csv(DATA_PATH)
texts = (df['subject'] + " " + df['relation'] + " " + df['object']).tolist()
total = len(texts)
print(f"📊 Total triplets: {total}")

# ✅ Setup FAISS
dimension = 384
index = faiss.IndexFlatL2(dimension)
metadata = []

# ✅ Resume if available
start_i = 0
if os.path.exists(FAISS_INDEX_PATH) and os.path.exists(METADATA_PATH):
    print("⏪ Resuming from previous...")
    index = faiss.read_index(FAISS_INDEX_PATH)
    with open(METADATA_PATH, 'rb') as f:
        metadata = pickle.load(f)
    start_i = len(metadata)
    print(f"🔁 Resuming from row {start_i}")

# ✅ Embedding
for i in tqdm(range(start_i, total, BATCH_SIZE), desc="🔁 Embedding..."):
    batch = texts[i:i + BATCH_SIZE]
    emb = model.encode(batch, convert_to_numpy=True, batch_size=BATCH_SIZE, device='cuda')
    index.add(emb)
    metadata.extend(batch)

    # Cleanup
    del emb
    gc.collect()
    torch.cuda.empty_cache()

# ✅ Save final index
faiss.write_index(index, FAISS_INDEX_PATH)
with open(METADATA_PATH, 'wb') as f:
    pickle.dump(metadata, f)

print(f"✅ Done! Total embedded: {len(metadata)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model loaded.
📊 Total triplets: 11758


🔁 Embedding...: 100%|██████████| 23/23 [00:13<00:00,  1.73it/s]

✅ Done! Total embedded: 11758


In [5]:
print("Total FAISS vectors:", index.ntotal)
print("Metadata entries   :", len(metadata))
print("Expected            :", len(df))  # should be 11758


Total FAISS vectors: 11758
Metadata entries   : 11758
Expected            : 11758


In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy as np

# 🔧 CONFIG
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
FAISS_INDEX_PATH = '/content/drive/MyDrive/kg_faiss_index.faiss'
METADATA_PATH = '/content/drive/MyDrive/kg_faiss_metadata.pkl'
TOP_K = 5  # how many results to show

# ✅ Load model
model = SentenceTransformer(MODEL_NAME).to('cuda')
print("✅ Model loaded.")

# ✅ Load index and metadata
index = faiss.read_index(FAISS_INDEX_PATH)
with open(METADATA_PATH, 'rb') as f:
    metadata = pickle.load(f)

print(f"✅ Loaded index with {index.ntotal} entries.")

# 🔎 Ask a query
query = "Which drug treats HIV infection?"
query_embedding = model.encode([query], convert_to_numpy=True, device='cuda')

# 🔁 Search
distances, indices = index.search(query_embedding, TOP_K)

# 📤 Show results
print("\n🔍 Top Results:")
for rank, idx in enumerate(indices[0]):
    print(f"{rank+1}. {metadata[idx]}  (Score: {distances[0][rank]:.4f})")


✅ Model loaded.
✅ Loaded index with 11758 entries.

🔍 Top Results:
1. APREPITANT treats HIV infection  (Score: 0.4416)
2. NEVIRAPINE treats HIV infection  (Score: 0.4682)
3. NEVIRAPINE treats AIDS  (Score: 0.5312)
4. NEVIRAPINE treats HIV-1 infection  (Score: 0.5639)
5. FAMOTIDINE treats HIV infection  (Score: 0.5669)
